In [3]:
# import classes from folder app/classes.py
from app.libs.classes import Shift, Nurse
import json


In [4]:
with open('/Users/user/Documents/HHCRSP/proyecto_HHCRSP/tmp/shifts.json', 'r') as f:
    shifts = json.load(f)

shifts = {shift['shift_id']: Shift(**shift) for shift in shifts}

with open('/Users/user/Documents/HHCRSP/proyecto_HHCRSP/tmp/nurses.json', 'r') as f:
    nurses = json.load(f)

nurses = {nurse['nurse_id']: Nurse(**nurse) for nurse in nurses}
nurses
# Additional parameters 

{51: Nurse(nurse_id=51, nurse_name='person_1', shift_preference='afternoon', accumulated_hours=-33.5, morning_availability_labor_day=1, morning_availability_weekend=1, afternoon_availability_labor_day=0, afternoon_availability_weekend=1, dates_off=[4], vacations=[]),
 42: Nurse(nurse_id=42, nurse_name='person_2', shift_preference='morning', accumulated_hours=-21.0, morning_availability_labor_day=1, morning_availability_weekend=1, afternoon_availability_labor_day=0, afternoon_availability_weekend=1, dates_off=[8, 15], vacations=[]),
 35: Nurse(nurse_id=35, nurse_name='person_3', shift_preference='afternoon', accumulated_hours=-33.5, morning_availability_labor_day=1, morning_availability_weekend=1, afternoon_availability_labor_day=0, afternoon_availability_weekend=1, dates_off=[8, 14, 21, 22, 28], vacations=[]),
 5: Nurse(nurse_id=5, nurse_name='person_4', shift_preference='afternoon', accumulated_hours=-21.5, morning_availability_labor_day=0, morning_availability_weekend=1, afternoon_av

In [5]:
DM = 8
HMM = 216
# Modelo

In [6]:
from datetime import datetime, timedelta
from math import floor
from typing import Dict

from app.libs.classes import Shift, Nurse
from app.libs.constants import HMM, DM
import pulp as plp
from itertools import product

In [7]:
import pulp as plp

model = plp.LpProblem("Nurse Scheduling", plp.LpMinimize)
I = set(nurses.keys())
J = set(shifts.keys())
K = set([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day for j in J])
week=set([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1] for j in J])

valid_keys = [(i, j) for (i,j) in product(I, J) if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day not in nurses[i].vacations]


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [8]:
X = plp.LpVariable.dicts("X", valid_keys, cat=plp.LpBinary)
W = plp.LpVariable.dicts("W", [(i, k) for (i,k) in product(I, K)], cat=plp.LpBinary)
Zmorning=plp.LpVariable.dicts("Zmorning", [(i, w) for (i,w) in product(I, week)], cat=plp.LpBinary)
Zafternoon=plp.LpVariable.dicts("Zafternoon", [(i, w) for (i,w) in product(I, week)], cat=plp.LpBinary)

V = plp.LpVariable.dicts("V", I, cat=plp.LpContinuous,lowBound=0)  # overtime
Y = plp.LpVariable.dicts("Y", I, cat=plp.LpContinuous,lowBound=0) # TOTAL NUMBER OF SHIFTS ASSIGNED TO NURSE i
MDH = plp.LpVariable("MDH", cat=plp.LpContinuous,lowBound=0) # MAXIMUM DIFFERENCE BETWEEN MAXIMUM SHIFTS AND REAL NUMBER OF SHIFTS ASSIGNED
DOff = plp.LpVariable.dicts("DOff", I, cat=plp.LpContinuous,lowBound=0) 


# Objective function

In [9]:
expr_PDL = plp.lpSum([(X[i, j] for (i,j) in valid_keys if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in nurses[i].dates_off)])
# accounts for the number of weekends a person is assigned to a shif not in his/her preference
expr_PWE = plp.lpSum([(X[i, j]) for (i,j) in valid_keys if
                        datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').weekday() in [5, 6] and
                        (nurses[i].shift_preference != shifts[j].shift_type)])

Obj_2 = MDH + plp.lpSum([V[i] for i in I]) * (1 / DM)

In [10]:
# accounts the number of shifts assigned to a nurse
for nurse in I:
    model+=Y[nurse] == plp.lpSum([X[nurse, shift] for shift in J if (nurse, shift) in valid_keys]), f"assigned_shifts_{nurse}"

In [11]:
 # accounts for the difference between shifts that should be assigned to a caregiver 
            # (considering overtime hours to be balanced) and real number of shifts assigned

for nurse in I:
    expr = Y[nurse] - (HMM - nurses[nurse].accumulated_hours) * (1 / DM) <= MDH
    model += expr, f"balance_hours_1_{nurses[nurse].nurse_id}"

    expr = (HMM - nurses[nurse].accumulated_hours) * (1 / DM) - Y[nurse]<= MDH
    model += expr,  f"balance_hours_2_{nurses[nurse].nurse_id}"
    

In [12]:
# dictionary enumerating the shifts happening in each day
shifts_per_day = {}
for shift in J:
    if shifts[shift].shift_date not in shifts_per_day:
        shifts_per_day[shifts[shift].shift_date] = []
    shifts_per_day[shifts[shift].shift_date].append(shift)


In [13]:
for i in I:
    for spd in shifts_per_day.keys():
        model += plp.lpSum([X[i, j] for j in shifts_per_day[spd] if (i, j) in valid_keys]) <= 1, f"one_shift_per_day_{nurses[i].nurse_id}_{spd}"


In [14]:
# accounts for overtime V hours required
for i in I:
    model += Y[i] * DM <= (HMM - nurses[i].accumulated_hours) + V[i], f"overtime_{nurses[i].nurse_id}"

In [15]:
# demand is satisfied
for j in J:
    model += plp.lpSum([X[(i, j)] for i in I if (i, j) in valid_keys]) >= shifts[j].demand, f"demand_{j}"
        

In [16]:
# for each weekend, a nurse works at most one day
for i in I:
    for j1 in J:
        if (i, j1) in valid_keys and datetime.strptime(shifts[j1].shift_date,'%Y-%m-%d').weekday() == 5:
            model += X[i, j1] + plp.lpSum([X[i, j2] for j2 in J if
                                                        (i, j2) in valid_keys and datetime.strptime(shifts[j2].shift_date, '%Y-%m-%d').weekday() == 6
                                                        and datetime.strptime(shifts[j2].shift_date,'%Y-%m-%d') == datetime.strptime(shifts[j1].shift_date, '%Y-%m-%d') + timedelta(days=1)]) <= 1, f"weekend_{nurses[i].nurse_id}_{j1}"


In [17]:
for i,k in product(I, K):
    model += (1 - plp.lpSum([X[i, j] for j in J if (i, j) in valid_keys and datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day == k])) ==W[i, k], f"day_is_off_{nurses[i].nurse_id}_{k}"

In [18]:
days = list([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day for j in J if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').weekday() in [5, 6]])

total_weekend_days = len(days)

for i in I:
    model += DOff[i] == plp.lpSum([W[i, k] for k in days]), f"weekend_off_1_{nurses[i].nurse_id}"
    model += DOff[i] >= floor(total_weekend_days / 2) + 1, f"weekend_off_2_{nurses[i].nurse_id}"

In [19]:
for i,w in product(I, week):
    model += Zmorning[i, w] + Zafternoon[i, w] <= 1, f"shifts_per_week_{nurses[i].nurse_id}_{w}"

In [20]:
# if a nurse works in a morning shift the varriable Z_morning is equal to 1
for i in I:
    for j in J:
        if (i, j) in valid_keys and shifts[j].shift_type == 'morning':
            model +=  X[i, j]<=Zmorning[i, datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1]], f"morning_shift_{nurses[i].nurse_id}_{j}"
        elif (i, j) in valid_keys and shifts[j].shift_type == 'afternoon':
            model +=  X[i, j]<=Zafternoon[i, datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1]], f"afternoon_shift_{nurses[i].nurse_id}_{j}"

In [21]:
#an afternoon shift cannot be followed by a morning shift
for i in I:
    for j in J:
        if (i, j) in valid_keys and shifts[j].shift_type == 'morning':
            model +=  X[i, j] + plp.lpSum([X[i, j2] for j2 in J if
                                                        (i, j2) in valid_keys and datetime.strptime(shifts[j2].shift_date, '%Y-%m-%d') == datetime.strptime(shifts[j].shift_date, '%Y-%m-%d') - timedelta(days=1)
                                                        and shifts[j2].shift_type == 'afternoon']) <= 1, f"morning_afternoon_{nurses[i].nurse_id}_{j}"

In [22]:
solver = plp.GUROBI_CMD(msg=1)
model.solve(solver)

Set parameter Username
Set parameter LogFile to value "gurobi.log"
Academic license - for non-commercial use only - expires 2024-04-04
Using license file /Users/user/gurobi.lic

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/fa9124716d0946a69ef5d7fb701f7f75-pulp.lp
Reading time = 0.01 seconds
OBJ: 2891 rows, 1666 columns, 11975 nonzeros

CPU model: Intel(R) Core(TM) i7-6820HQ CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2891 rows, 1666 columns and 11975 nonzeros
Model fingerprint: 0x2d96f344
Variable types: 26 continuous, 1640 integer (1640 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 1566 rows and 558 columns
Presolve time: 0.03s
Presolved: 1

0

In [23]:
import numpy as np
model+= expr_PDL+expr_PWE<=np.inf, "value_obj_1"
model+= Obj_2<=np.inf , "value_obj_2"
#
model.setObjective(expr_PDL+expr_PWE)
# solve the problem using HIGHS
solver = plp.GUROBI_CMD(msg=0)
model.solve(solver)
best_1=expr_PDL.value()+expr_PWE.value()
ctr = model.constraints["value_obj_1"]
ctr.changeRHS(best_1 )
model.setObjective(Obj_2)
model.solve(solver)
worst_2=Obj_2.value()
ctr = model.constraints["value_obj_1"]
ctr.changeRHS(np.inf)
ctr = model.constraints["value_obj_2"]
ctr.changeRHS(np.inf) 
model.setObjective(Obj_2)
model.solve(solver)
best_2=Obj_2.value()
ctr = model.constraints["value_obj_2"]
ctr.changeRHS(best_2 )

model.setObjective(expr_PDL+expr_PWE)
model.solve(solver)
worst_1=expr_PDL.value()+expr_PWE.value()
print(best_1,best_2,worst_1,worst_2)

ctr = model.constraints["value_obj_1"]
ctr.changeRHS(best_1)
ctr = model.constraints["value_obj_2"]
ctr.changeRHS(np.inf)
model.setObjective(Obj_2)




TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [ ]:

epsilon = 1
model.solve()

while model.status == 1:
    model.solve()
    print(expr_PDL.value()+expr_PWE.value(),Obj_2.value())
    epsilon += 1
    ctr = model.constraints["value_obj_1"]
    ctr.changeRHS(best_1 + epsilon)


    if best_1 + epsilon > worst_1:
        break
    
model.setObjective(Obj_2)

6.0 4.625
8.0 2.625
9.0 2.5625
10.0 2.5625
11.0 1.625
12.0 1.5625


In [ ]:
#Get gurobi model status code
#set gurobi verbose to 1
solver = plp.GUROBI_CMD(msg=0)
model.solve(solver)

1

In [ ]:
import pandas as pd
import numpy as np
"""for i in I:
    for j in J:
        if (i, j) in valid_keys and X[i, j].varValue > 0.5:
            if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in [1,7,8,14,15,21,22,28,29] and shifts[j].shift_type!=nurses[i].shift_preference:
                print(f"Nurse {nurses[i].nurse_name} works in shift {shifts[j].shift_id} on {shifts[j].shift_date} ({shifts[j].shift_type}--{shifts[j].shift}--{nurses[i].shift_preference}) with vacation {nurses[i].vacations} and days off {nurses[i].dates_off}.")
            if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in nurses[i].dates_off:
                print(f"Nurse {nurses[i].nurse_name} works in shift {shifts[j].shift_id} on {shifts[j].shift_date} ({shifts[j].shift_type}--{shifts[j].shift}--{nurses[i].shift_preference}) with vacation {nurses[i].vacations} and days off {nurses[i].dates_off}.")
"""
date_range=np.sort(list(set([shifts[j].shift_date for j in range(len(shifts))])))
nurses_shifts = {nurses[i].nurse_name: {shifts[j].shift_date:shifts[j].shift+" - "+shifts[j].shift_type for j in J if (i,j) in valid_keys and X[i, j].varValue > 0.5} for i in I}          
nurses_morning_shifts={nurses[i].nurse_name: {shifts[j].shift_date:shifts[j].shift for j in J if (i,j) in valid_keys and X[i, j].varValue > 0.5 and shifts[j].shift_type=="morning"} for i in I}
nurses_afternoo_shifts={nurses[i].nurse_name: {shifts[j].shift_date:shifts[j].shift for j in J if (i,j) in valid_keys and X[i, j].varValue > 0.5 and shifts[j].shift_type=="afternoon"} for i in I}

tabulated_shifts=[]
for nurse in nurses_shifts:
    line=[nurse]
    for date in date_range:
        if date not in nurses_shifts[nurse]:
            line.append("-")
            line.append("-")
        elif date in nurses_morning_shifts[nurse]:
           line.append(nurses_morning_shifts[nurse][date])
           line.append("-")
        else:
           line.append("-")
           line.append(nurses_afternoo_shifts[nurse][date])
    tabulated_shifts.append(line)
tabulated_shifts=pd.DataFrame(tabulated_shifts, columns=["Nurse"]+[str(date)+str(jornada) for date,jornada in product(date_range,["-mañana","-tarde"])])

In [ ]:

tabulated_shifts.to_excel("tabulated_shifts.xlsx")
tabulated_shifts

,Nurse,2022-05-01-mañana,2022-05-01-tarde,2022-05-02-mañana,2022-05-02-tarde,2022-05-03-mañana,2022-05-03-tarde,2022-05-04-mañana,2022-05-04-tarde,2022-05-05-mañana,...,2022-05-27-mañana,2022-05-27-tarde,2022-05-28-mañana,2022-05-28-tarde,2022-05-29-mañana,2022-05-29-tarde,2022-05-30-mañana,2022-05-30-tarde,2022-05-31-mañana,2022-05-31-tarde
0,person_5,M,-,COM,-,COM,-,COM,-,COM,...,COM,-,-,-,COM,-,-,T1,-,CHX
1,person_6,DISP,-,-,T1,-,CHX,-,T2,-,...,COM,-,COM,-,-,-,-,T1,-,T2
2,person_2,-,T1,-,-,M,-,COM,-,-,...,-,-,-,-,DISP,-,M,-,COM,-
3,person_7,-,-,M,-,M,-,COM,-,M,...,-,CHX,-,T1,-,-,-,T1,-,T1
4,person_13,-,-,M,-,COM,-,M,-,COM,...,M,-,-,-,M,-,COM,-,M,-
5,person_8,-,-,COM,-,-,-,M,-,COM,...,-,T2,-,-,-,-,COM,-,-,-
6,person_14,M,-,-,-,COM,-,M,-,M,...,COM,-,-,-,M,-,-,T1,-,T1
7,person_10,-,T1,-,CHX,-,T1,-,T1,-,...,-,T2,-,-,-,T1,-,T1,-,T1
8,person_1,-,-,-,T1,-,CHX,-,T2,-,...,-,CHX,-,T1,-,-,-,T1,-,T1
9,person_3,-,-,M,-,COM,-,COM,-,-,...,-,T1,-,-,-,-,-,T1,-,CHX
